# Agents and Tools

What are “Agents & Tools”?
- Tools are functions (APIs, retrievers, calculators) the model can call.
- Agents are LLM-powered planners that decide dynamically which tool(s) to call, in what order, based on the user’s request.

They are useful when tasks require external actions, multi-step reasoning, or dynamic selection (e.g., “Search Wikipedia and then summarize”).

We’ll cover:

- Defining simple Tools (calculator, search stub)
- Using an LCEL tool manually
- A lightweight Agent that calls tools with create_openai_functions_agent (v0.3 approach)
- Understanding tool-calling JSON
- Adding memory to agents (brief mention)

Run this bootstrap cell before running subsequent cells

In [ ]:
import os, math, time
from dotenv import load_dotenv
load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("Missing OPENAI_API_KEY in .env")

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain.agents import create_openai_functions_agent, tool as agent_tool, AgentExecutor

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
print("✅ Ready: Chat model for agents & tools")

## Defining Tools (functions the agent can call)

In LangChain v0.3, tools are simply Python callables decorated with `@tool`

In [ ]:
# A simple calculator tool
@tool
def calculator(expression: str) -> str:
    """Evaluates basic math expressions like '2 + 2 * 5'."""
    print("Calc. is used")
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

# A fake search tool (simulate an API)
@tool
def search_web(query: str) -> str:
    """Search the web and return a short result (simulated)."""
    print("Search is used")
    fake_results = {
        "LangChain": "LangChain is a framework for building LLM apps.",
        "MMR": "Maximal Marginal Relevance balances relevance & diversity."
    }
    return fake_results.get(query, f"No results found for '{query}'")

tools = [calculator, search_web]

print(calculator.invoke("5 * (3 + 2)"))
print(search_web.invoke("LangChain"))

## Creating Agents in OpenAI

LangChain v0.3 provides `create_openai_functions_agent` which leverages the model’s function/tool calling abilities.

Here's an agent that uses the calculator and search_web tools.

In [ ]:
# Create a prompt template for the agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools when needed."),
    ("user", "{input}"),
    MessagesPlaceholder("agent_scratchpad")
])

# Create the agent with tool definitions
agent = create_openai_functions_agent(llm, tools, prompt)

# Wrap agent into an executor (for run + verbose logs)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Ask a multi-step query:
query = "What's 2+2*5, then tell me what LangChain is?"
result = executor.invoke({"input": query})
print("\n--- Agent Final Output ---")
print(result["output"])

### How it works

- `MessagesPlaceholder("agent_scratchpad")`: Critical piece! This is where the agent keeps track of intermediate reasoning steps and tool invocations. The functions agent writes “thoughts” and tool calls here while working out the answer.
- `create_openai_functions_agent`: Special constructor that wires up:
    - The LLM (as a “planner” that decides what to do).
    - The tools (actions it can take).
    - The prompt (rules + placeholders for scratchpad).
- The `AgentExecutor` is the runtime wrapper. It manages:
    - Passing inputs into the agent.
    - Handling intermediate steps (showing reasoning if verbose=True).
    - Returning the final output.